In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
df=pd.read_csv("C:\\Users\\thris\\Desktop\\cleaned_loan_data.csv")
df.head(2)

NameError: name 'pd' is not defined

In [52]:
df.columns

Index(['ID', 'year', 'loan_limit', 'Gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'Credit_Worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'Interest_rate_spread', 'Upfront_charges', 'term', 'Neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'income', 'credit_type', 'Credit_Score', 'co-applicant_credit_type',
       'age', 'submission_of_application', 'LTV', 'Region', 'Security_Type',
       'Status', 'dtir1'],
      dtype='object')

In [53]:
df.shape

(148670, 34)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [55]:
df.isnull().sum()

ID                               0
year                             0
loan_limit                    3344
Gender                           0
approv_in_adv                  908
loan_type                        0
loan_purpose                   134
Credit_Worthiness                0
open_credit                      0
business_or_commercial           0
loan_amount                      0
rate_of_interest             36439
Interest_rate_spread         36639
Upfront_charges              39642
term                            41
Neg_ammortization              121
interest_only                    0
lump_sum_payment                 0
property_value               15098
construction_type                0
occupancy_type                   0
Secured_by                       0
total_units                      0
income                        9150
credit_type                      0
Credit_Score                     0
co-applicant_credit_type         0
age                            200
submission_of_applic

In [56]:
df.duplicated().sum()

0

FILL CATEGORICAL DATA

In [57]:
categorical_cols = ['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose',
                    'Credit_Worthiness', 'open_credit', 'business_or_commercial',
                    'Neg_ammortization', 'interest_only', 'lump_sum_payment',
                    'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
                    'credit_type', 'co-applicant_credit_type', 'age',
                    'submission_of_application', 'Region', 'Security_Type']


for col in categorical_cols:
    print(f"{col}: {df[col].isna().sum()}")

loan_limit: 3344
Gender: 0
approv_in_adv: 908
loan_type: 0
loan_purpose: 134
Credit_Worthiness: 0
open_credit: 0
business_or_commercial: 0
Neg_ammortization: 121
interest_only: 0
lump_sum_payment: 0
construction_type: 0
occupancy_type: 0
Secured_by: 0
total_units: 0
credit_type: 0
co-applicant_credit_type: 0
age: 200
submission_of_application: 200
Region: 0
Security_Type: 0


In [58]:
df['loan_limit'] = df['loan_limit'].fillna(df['loan_limit'].mode()[0])
df['approv_in_adv'] = df['approv_in_adv'].fillna(df['approv_in_adv'].mode()[0])
df['loan_purpose'] = df['loan_purpose'].fillna(df['loan_purpose'].mode()[0])
df['Neg_ammortization'] = df['Neg_ammortization'].fillna(df['Neg_ammortization'].mode()[0])
df['submission_of_application'] = df['submission_of_application'].fillna(df['submission_of_application'].mode()[0])
df['age'] = df['age'].fillna(df['age'].mode()[0])

In [59]:
df["Gender"].value_counts()

Gender
Male                 42346
Joint                41399
Sex Not Available    37659
Female               27266
Name: count, dtype: int64

In [62]:
df["Gender"]=df["Gender"].replace("Sex Not Available",df["Gender"].mode()[0])

USE LABEL ENCODER

In [64]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for col in categorical_cols:
    df[col]=le.fit_transform(df[col])

FILLING MISSING VALUES

Rate of interest and Interest_rate_spread

In [66]:
df.corr()["rate_of_interest"].sort_values(ascending=False)

rate_of_interest             1.000000
Interest_rate_spread         0.614908
term                         0.209330
open_credit                  0.191423
Credit_Worthiness            0.153968
submission_of_application    0.141390
total_units                  0.082623
approv_in_adv                0.057598
dtir1                        0.055124
business_or_commercial       0.051474
age                          0.032743
loan_limit                   0.029347
Status                       0.022957
lump_sum_payment             0.015249
interest_only                0.009160
ID                           0.000442
LTV                         -0.000220
Credit_Score                -0.001331
Region                      -0.012937
credit_type                 -0.025887
Gender                      -0.026391
income                      -0.041809
co-applicant_credit_type    -0.043587
Upfront_charges             -0.076473
property_value              -0.122613
loan_amount                 -0.150844
Neg_ammortiz

In [68]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import numpy as np

# Impute temporary missing values for training
imputer = SimpleImputer(strategy='mean')

# Prepare data for 'rate_of_interest'
roi_not_missing = df[df['rate_of_interest'].notna()]
roi_missing = df[df['rate_of_interest'].isna()]
irs_temp_imputed = imputer.fit_transform(roi_not_missing[['Interest_rate_spread']])

# Train model to predict 'rate_of_interest' using 'Interest_rate_spread'
model_irs_to_roi = LinearRegression()
model_irs_to_roi.fit(irs_temp_imputed, roi_not_missing['rate_of_interest'])

# Predict missing 'rate_of_interest'
df.loc[df['rate_of_interest'].isna(), 'rate_of_interest'] = model_irs_to_roi.predict(
    imputer.transform(roi_missing[['Interest_rate_spread']])
)

# Prepare data for 'Interest_rate_spread'
irs_not_missing = df[df['Interest_rate_spread'].notna()]
irs_missing = df[df['Interest_rate_spread'].isna()]
roi_temp_imputed = imputer.fit_transform(irs_not_missing[['rate_of_interest']])

# Train model to predict 'Interest_rate_spread' using 'rate_of_interest'
model_roi_to_irs = LinearRegression()
model_roi_to_irs.fit(roi_temp_imputed, irs_not_missing['Interest_rate_spread'])

# Predict missing 'Interest_rate_spread'
df.loc[df['Interest_rate_spread'].isna(), 'Interest_rate_spread'] = model_roi_to_irs.predict(
    imputer.transform(irs_missing[['rate_of_interest']])
)


upfront charges

In [70]:
df.corr()["Upfront_charges"].sort_values(ascending=False)

Upfront_charges              1.000000
submission_of_application    0.334916
business_or_commercial       0.151732
age                          0.101647
Gender                       0.068344
loan_amount                  0.065556
Region                       0.062290
property_value               0.053043
total_units                  0.050819
loan_limit                   0.043156
Interest_rate_spread         0.032761
Neg_ammortization            0.023771
approv_in_adv                0.018524
income                       0.016580
lump_sum_payment             0.009544
loan_purpose                 0.009300
co-applicant_credit_type     0.004481
interest_only                0.000746
dtir1                        0.000115
Credit_Score                -0.001484
ID                          -0.005507
Credit_Worthiness           -0.006602
credit_type                 -0.014505
Status                      -0.019138
LTV                         -0.031347
occupancy_type              -0.046536
term        

In [71]:
df=df.drop("Upfront_charges",axis=1)

term

In [72]:
df["term"]=df["term"].fillna(360)

Neg_ammortization

In [73]:
df.corr()["Neg_ammortization"].sort_values(ascending=False)

Neg_ammortization            1.000000
approv_in_adv                0.077195
loan_purpose                 0.067128
loan_amount                  0.027458
property_value               0.027078
term                         0.021358
interest_only                0.018190
occupancy_type               0.015302
business_or_commercial       0.014391
Security_Type                0.006932
construction_type            0.006932
loan_type                    0.005984
income                       0.003777
ID                           0.001185
Region                      -0.000537
Credit_Score                -0.000958
open_credit                 -0.001976
loan_limit                  -0.006279
Gender                      -0.006317
Secured_by                  -0.006932
credit_type                 -0.007728
LTV                         -0.008876
co-applicant_credit_type    -0.010603
age                         -0.012946
total_units                 -0.013450
dtir1                       -0.019885
Interest_rat

In [75]:
df["Neg_ammortization"]=df["Neg_ammortization"].fillna(df["Neg_ammortization"].mode()[0])

LTV

In [77]:
df["LTV"]=df["LTV"].fillna(df["LTV"].mean())

DTIR1

In [78]:
df["dtir1"]=df["dtir1"].fillna(df["dtir1"].mean())

income

In [80]:
df["income"]=df["income"].fillna(df["income"].mean())

property_value

In [82]:
df["property_value"]=df["property_value"].fillna(df["property_value"].mean())

In [83]:
df.isna().sum().sort_values(ascending=False)

ID                           0
property_value               0
Status                       0
Security_Type                0
Region                       0
LTV                          0
submission_of_application    0
age                          0
co-applicant_credit_type     0
Credit_Score                 0
credit_type                  0
income                       0
total_units                  0
Secured_by                   0
occupancy_type               0
construction_type            0
lump_sum_payment             0
year                         0
interest_only                0
Neg_ammortization            0
term                         0
Interest_rate_spread         0
rate_of_interest             0
loan_amount                  0
business_or_commercial       0
open_credit                  0
Credit_Worthiness            0
loan_purpose                 0
loan_type                    0
approv_in_adv                0
Gender                       0
loan_limit                   0
dtir1   

In [85]:
df.to_csv("C:\\Users\\thris\\Desktop\\cleaned_loan_data.csv", index=False)